In [1]:
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
import matplotlib.pyplot as plt
import numpy as np

def gen_matrix(image, nb_channels, random_state):
      matrix_A = np.array([[0, -1, 0],
                           [-1, 4, -1],
                           [0, -1, 0]])
      matrix_B = np.array([[0, 0, 0],
                          [0, -4, 1],
                          [0, 2, 1]])
      if random_state.rand() < 0.5:
          return [matrix_A] * nb_channels
      else:
          return [matrix_B] * nb_channels
# aug = iaa.Convolve(matrix=gen_matrix)

seq = iaa.Sequential([
    ## unsure if Crop is likely to be useful offsett may be better
#     iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    ## definitely worth doing Fliplr as it doubles the size.
#     iaa.Fliplr(0.5), # horizontally flip 50% of the images
    ## GaussianBlur is also worth performing on for generalising
    ## possibly ignore it for my own data?  
#     iaa.GaussianBlur(sigma=(0, 3.0)), # blur images with a sigma of 0 to 3.0
    ## Noise is also worth doing for generalising.
#     iaa.AdditiveGaussianNoise(scale=0.1*255) #looks like static
#     iaa.EdgeDetect(alpha=(0.0, 1.0))
    
    ## I think convolve is probably useful if you define it with the sharpness to demonstrate
    ## the advantages of using Convolve is to view what the Computer might see.
    iaa.Convolve(matrix=gen_matrix),
    ## sharpeness definitely feels useful for defining the edges of the area.
#     iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0))
    ## probably worth doing sometimes, it might be better with 
#     iaa.ContrastNormalization((0.5, 1.5))
#     iaa.ContrastNormalization((0.5, 1.5), per_channel=0.5),
    ## Affine scale
    ## might be an issue due to where the signs are typically located don't increase the sign beyond 1.0
    ## unless you write a function to discard bounding boxes outside the actual area.
#     iaa.Affine(scale=(0.35, .90)),
    ## scale indepently on each axis indepently changing axis, might be really useful
#     iaa.Affine(scale={'x': (0.5, 1), 'y': (0.35, 1.0)}),
    ## this could be useful however, it would require it the translation to not cut off the signs in the images
    ## possible function to ensure it's not missing
    ## note though it's on percent
#     iaa.Affine(translate_percent={'x': (-0.02, 0.2), 'y': (-0.2, 0.2)})
    ## note the below is a pixel translation which might actually be better due to it being more minor
#     iaa.Affine(translate_pixel={'x': (-20, 20), 'y': (-20, 20)})
    ## note that this will be worth while if it's using with scaling and/or doesn't cut off
    ## the actual sign
#     iaa.Affine(rotate=(-45, 45))
    ## shear is definitely worth including in the data augmentation process
#     iaa.Affine(shear=(-160,160))
    ## unsure of what the mode kwarg is doing currently
#     iaa.Affine(translate_percent={"x": (-0.20)}, mode=ia.ALL, cval=(0, 255))
    ## Really useful on low level distortion
#     iaa.PiecewiseAffine(scale=(0.008, 0.0095))
    ## use ElasticTransformation to add random can pass in tuples for randomness
    ## to the distortion field
    iaa.ElasticTransformation(alpha=(0, 5.0), sigma=(0.25, 1))

])
img1 = cv2.imread('/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_235.jpg')

img = cv2.imread('/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_105.jpg')
# cv2.imshow('test', img)
imgs = [img, img1]
print(type(img))
# seq.show_grid(imgs[0], cols=8, rows=8)
# cropped = img[7:100, 22:67]
# # cv2.imshow('cropped', cropped)
# # cv2.waitKey(0)

img_aug = seq.augment_image(img)
# seq.show_grid(img_aug, cols=8, rows=8)
# print(img.shape)
# print(img_aug.shape)
# print(len(img_aug))

# cv2.imshow('aug??', img_aug)
# cv2.waitKey(0)

# # for batch_idx in range(1000):
# #     # 'images' should be either a 4D numpy array of shape (N, height, width, channels)
# #     # or a list of 3D numpy arrays, each having shape (height, width, channels).
# #     # Grayscale images must have shape (height, width, 1) each.
# #     # All images must have numpy's dtype uint8. Values are expected to be in
# #     # range 0-255.
# #     images = load_batch(batch_idx)
# #     images_aug = seq.augment_images(images)
# #     train_on_images(images_aug)

<class 'numpy.ndarray'>


# Write out one image as new.

1. It needs the path of the image.
2. The bounding boxes of the image.
3. Write the number 

In [2]:
import os
import xml.etree.ElementTree as ET
import cv2

base_dir = '/Users/datascience4/Documents/training3'
print(len(os.listdir(base_dir)))
xml_files = os.listdir(base_dir)
xml_files = [x for x in os.listdir(base_dir) if '.xml' in x]
print(xml_files)
items = []

## come back to this

#let's read the first xml
test_xml = os.path.join(base_dir, xml_files[0])
tree = ET.parse(test_xml)
root = tree.getroot()
## path can be used for cv2 read image
path = root.find('path').text
print(path)
## find the object and xmin, ymin, xmax, ymax
for child in root.iter('object'):
    if child.find('bndbox'):
        bndbox = child.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        
print(xmin, ymin, xmax, ymax)

# ia.seed(1)
## pass the path into cv2
image = cv2.imread(path)
# cv2.imshow('path', image)
# cv2.waitKey(0)

## bounding boxes now for the image takes a list
bbs = ia.BoundingBoxesOnImage([
    ia.BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax)
], shape=image.shape)

seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect bb
    iaa.Affine(
        translate_px={"x": 40, "y": 60},
#         scale=(0.5, 0.7),
        rotate=(-25, 25)
        
    ) # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
])

seq = iaa.Sequential([
    iaa.Fliplr(0.5), #horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    #Small gaussian blur with random sigma between 0 and 0.5
    # But we only blur about 50% of all images.
    iaa.Sometimes(0.5,
                 iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    #Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    #Add gaussian noise.
    #For 50% of all images, we sample the noise once per pixel..
    #For the other 50% of all images, we sample the noise per pixel
    #AND channel. This can change the color (not only the brightness)
    #of the pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255),
                             per_channel=0.5),
    #Make some images brighter and some darker.
    #in 20% of all cases we sample the multiplier once per channel.
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    #Apply affine transformations to each image.
    #Scale/zoom them, translate/move them, rotate them, and shear them.
    iaa.Affine(
        scale={'x': (0.8, 1.2), 'y': (0.8, 1.2)},
        translate_percent={'x': (-0.2, 0.2), 'y': (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) #apply augmenters in random order

## make our sequence deterministic.
# we can now apply it to the image and then to the bbs and it will
# lead to the same augmentations
# IMPORTANT: call this once PER BATCH, otherwise you will always
# get the exactly same augmentations for every batch!
seq_det = seq.to_deterministic()

##Augment bbs and images.
## as we only have one image and list of bbs we use
## [image] and [bbs] to turn both into lists (batches) for the
## functions and then [0] to reverse that. In a real experiment, your
## variables would likely already be lists.
image_aug = seq_det.augment_images([image])[0]
bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

##print the coordinates before/after augmentation(see below)
## use .x1_int, y1_int, ... to get interger coordinates
for i in range(len(bbs.bounding_boxes)):
    before = bbs.bounding_boxes[i]
    after = bbs_aug.bounding_boxes[i]
    print(f"BB {i}:, ({before.x1, before.y1, before.x2, before.y2} \
          -> ({after.x1, after.y1, after.x2, after.y2})")
    
# image with bbs before/after augmentation (shown below)
image_before = bbs.draw_on_image(image, thickness=2)
# cv2.imshow('image before', image_before)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

image_after = bbs_aug.draw_on_image(image_aug, thickness=2, color=[0, 0, 255])
# cv2.imshow('image after', image_after)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

cv2.imwrite('/Users/datascience4/Desktop/test1.png', image_after)

385
['A3400_NB1_RAV_R07_180516084515_frame_3598.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2862.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2447.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2335.xml', 'A3400_NB1_RAV_R07_180516084515_frame_4022.xml', 'A3400_NB1_RAV_R07_180516084515_frame_224.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2255.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2527.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3163.xml', 'A3400_NB1_RAV_R07_180516084515_frame_350.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3375.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3374.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3837.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3162.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2240.xml', 'A3400_NB1_RAV_R07_180516084515_frame_225.xml', 'A3400_NB1_RAV_R07_180516084515_frame_4023.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2446.xml', 'A3400_NB1_RAV_R07_180516084515_frame_2863.xml', 'A3400_NB1_RAV_R07_180516084515_frame_3599.xml', 'A3400_NB1_RAV_R07

True

## Things to do
1. Create a folder for the augmented images
2. Then run through all images (drawing on each image the bounding box)
3. Sense check that all are in the bounding boxes.

In [3]:
try:
    os.mkdir('/Users/datascience4/Documents/training_3_data_aug')
except FileExistsError:
    print('folder already exists')

folder already exists


In [4]:
aug_dir = '/Users/datascience4/Documents/training_3_data_aug'

## SECTION DEALS WITH BOUNDING BOXES NOT IN THE IMAGE
## funtion to pad the image and deal with bounding boxes not in the images

# Pad image with a 1px white and (BY-1)px black border
def pad(image, by):
    if by <= 0:
        return image
    image_border1 = np.pad(
        image, ((1, 1), (1, 1), (0, 0)),
        mode="constant", constant_values=255
    )
    image_border2 = np.pad(
        image_border1, ((by-1, by-1), (by-1, by-1), (0, 0)),
        mode="constant", constant_values=0
    )
    return image_border2

# Draw BBs on an image
# and before doing that, extend the image plane by BORDER pixels.
# Mark BBs inside the image plane with green color, those partially inside
# with orange and those fully outside with red.
def draw_bbs(image, bbs, border):
    ##rgb in open cv isn't rgb but bgr - it's bad for that!
    GREEN = (0, 255, 0)
    ORANGE = (0, 140, 255)
    RED = (0, 0, 255)
    image_border = pad(image, border)
    for bb in bbs.bounding_boxes:
        if bb.is_fully_within_image(image.shape):
            color = GREEN
        elif bb.is_partly_within_image(image.shape):
            print('partly?')
            color = ORANGE
        else:
            print('not in at all')
            color = RED
        image_border = bb.shift(left=border, top=border).draw_on_image(image_border, thickness=2, color=color)
    return image_border

def draw_bbs_count(image, bbs, border, **kwargs):
    counter = kwargs.get('counter', None)
    ## using open CV2 so colours are in BGR and not RGB so check the order if it comes different
    GREEN = (0, 255, 0)
    ORANGE = (0, 140, 255)
    RED = (0, 0, 255)
    ## probably can remove the ine beloe as it won't be needed
    image_border = pad(image, border)
    for bb in bbs.bounding_boxes:
        if bb.is_fully_within_image(image.shape):
            color = GREEN
            counter += 1
        elif bb.is_partly_within_image(image.shape):
            print('partly version 2\n\n\n')
            color = ORANGE
            image_border = None
        else:
            print('not in at all\n\n\n')
            color = RED
            image_border = None
        if image_border is not None:
            image_border = bb.shift(left=border, top=border).draw_on_image(image_border, thickness=2, color=color)
    return image_border, counter
    
    

## write out a file that makes ue of similar layout as draw_bbs



## attempt to write out the same image multiple times in different ways
for i in range(10):
    #let's read the first xml
    test_xml = os.path.join(base_dir, xml_files[i])
    print(i)
    tree = ET.parse(test_xml)
    root = tree.getroot()
    ## path can be used for cv2 read image
    path = root.find('path').text
    print(path)
    ## find the object and xmin, ymin, xmax, ymax
    for child in root.iter('object'):
        if child.find('bndbox'):
            bndbox = child.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
    
    image = cv2.imread(path)
    bbs = ia.BoundingBoxesOnImage([
                    ia.BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax)
    ], shape=image.shape)
    ## new
    counter = 0
    ## Set the counter to less than the amount of images you actually want, so if it's 50 of each individual image
    while counter < 50:
#         counter += 1
        print(counter)
        seq_det = seq.to_deterministic()
        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
        print('bbs_aug', bbs_aug.bounding_boxes)
        print('type', type(bbs_aug.bounding_boxes))
        ## take the bbs_aug, and . then measure it on the image_aug if the image is fully black it's wrong
        bbs_aug_array = np.array([
            bbs_aug.bounding_boxes[0].y1,
            bbs_aug.bounding_boxes[0].y2,\
            bbs_aug.bounding_boxes[0].x1,\
            bbs_aug.bounding_boxes[0].x2])
        bbs_aug_array = bbs_aug_array.astype(int)
        image_aug_cropped = image_aug[bbs_aug_array[0]:bbs_aug_array[1], bbs_aug_array[2]:bbs_aug_array[3]]
#         is_black = np.array([0,0,0], dtype='uint8')
#         is_quite_black = np.array([20, 20, 20], dtype='uint8')
#         mask = cv2.inRange(image_aug_cropped, is_black, is_quite_black)
#         output = cv2.bitwise_and(image_aug_cropped, image_aug_cropped, mask = mask)
#         output1 = output.copy()
#         output1[mask == 0] = (255, 255, 255)
        ## convert image_aug_cropped to grayscale
        image_aug_cropped_grayscale = cv2.cvtColor(image_aug_cropped, cv2.COLOR_BGR2GRAY)
        if cv2.countNonZero(image_aug_cropped_grayscale) < 50:
            print("Image is black")
#             cv2.imshow('crop1', image_aug_cropped_grayscale)
#             cv2.waitKey(0)
        else:
            print('Colored image')
        
        print('bbs_aug', bbs_aug)
#         is_black = np.array([0,0,0], dtype="uint8")
#             print(type(is_black))
#             bb_array = np.array([bb.y1, bb.y2, bb.x1, bb.x2])
#             bb_array = bb_array.astype(int)
#             print(bb_array)
# #             mask = cv2.inRange(image, is_black, is_black)
# #             output = cv2.bitwise_and(image, image, mask = mask)
# #             # show the images
# #             cv2.imshow("images", np.hstack([image, output]))
# #             cv2.waitKey(0)
#             cropped = image_border[bb_array[0]: bb_array[1], bb_array[2], bb_array[3]]
#             cv2.imshow('crop 1', cropped)
#             cv2.waitKey(0)
        image_after, counter = draw_bbs_count(image_aug, bbs_aug, 1, counter=counter)
        if image_after is not None:
            cv2.imwrite(os.path.join(aug_dir, ('bb_count_photo' + str(i) + 'counter_no'\
                                                + str(counter) + '.jpeg')), image_after)
        
    ##
#     for j in range(10):
#         seq_det = seq.to_deterministic()
#         image_aug = seq_det.augment_images([image])[0]
#         bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
#         ## test removing
#         image_after = bbs_aug.draw_on_image(image_aug, thickness=2, color=[255, 140, 0])
#         ## testing
#         image_after1 = draw_bbs(image_aug, bbs_aug, 100, j)
#         cv2.imwrite(os.path.join(aug_dir, ('bb' + str(i) + str(j) + '.png')), image_after1)
# #         image_after2 = draw_bbs(image_aug, bbs_aug.remove_out_of_image(), 100)
# #         cv2.imwrite(os.path.join(aug_dir, ('bb_removed' + str(i) + str(j) + '.png')), image_after2)
# #         image_after3 = draw_bbs(image_aug, bbs_aug.remove_out_of_image()\
# #                 .cut_out_of_image(), 100)
# #         cv2.imwrite(os.path.join(aug_dir, ('bb_removed_and_cut_out' + str(i) + str(j) + '.png')), image_after3)
# #         cv2.imwrite(os.path.join(aug_dir, (str(i) + str(j) + '.png')), image_after)
#         print(bbs, bbs_aug)


0
/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_3598.jpg
0
bbs_aug [BoundingBox(x1=745.9010, y1=140.8647, x2=808.8901, y2=204.1041, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=745.9010, y1=140.8647, x2=808.8901, y2=204.1041, label=None)], shape=(768, 1360, 3))
1
bbs_aug [BoundingBox(x1=597.9884, y1=89.1121, x2=678.0499, y2=138.9177, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=597.9884, y1=89.1121, x2=678.0499, y2=138.9177, label=None)], shape=(768, 1360, 3))
2
bbs_aug [BoundingBox(x1=941.4210, y1=213.0634, x2=1015.4062, y2=294.1096, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=941.4210, y1=213.0634, x2=1015.4062, y2=294.1096, label=None)], shape=(768, 1360, 3))
3
bbs_aug [BoundingBox(x1=788.8561, y1=-0.3780, x2=851.9414, y2=66.8580, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBo

bbs_aug [BoundingBox(x1=553.6238, y1=315.4881, x2=630.4390, y2=377.8430, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=553.6238, y1=315.4881, x2=630.4390, y2=377.8430, label=None)], shape=(768, 1360, 3))
32
bbs_aug [BoundingBox(x1=520.4125, y1=75.7854, x2=575.7868, y2=121.7193, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=520.4125, y1=75.7854, x2=575.7868, y2=121.7193, label=None)], shape=(768, 1360, 3))
33
bbs_aug [BoundingBox(x1=765.5112, y1=151.8334, x2=836.3927, y2=202.5372, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=765.5112, y1=151.8334, x2=836.3927, y2=202.5372, label=None)], shape=(768, 1360, 3))
34
bbs_aug [BoundingBox(x1=893.7875, y1=261.2144, x2=969.7816, y2=335.8614, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=893.7875, y1=261.2144, x2=969.7816, y2=335.8614, label=None)], shape=(768, 136

bbs_aug [BoundingBox(x1=351.6669, y1=27.8091, x2=381.6361, y2=57.9102, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=351.6669, y1=27.8091, x2=381.6361, y2=57.9102, label=None)], shape=(768, 1360, 3))
14
bbs_aug [BoundingBox(x1=776.3678, y1=187.8098, x2=819.1281, y2=220.7984, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=776.3678, y1=187.8098, x2=819.1281, y2=220.7984, label=None)], shape=(768, 1360, 3))
15
bbs_aug [BoundingBox(x1=1101.1706, y1=111.9758, x2=1130.2319, y2=141.4036, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1101.1706, y1=111.9758, x2=1130.2319, y2=141.4036, label=None)], shape=(768, 1360, 3))
16
bbs_aug [BoundingBox(x1=830.7687, y1=153.2329, x2=864.4161, y2=186.5258, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=830.7687, y1=153.2329, x2=864.4161, y2=186.5258, label=None)], shape=(768, 1

bbs_aug [BoundingBox(x1=970.5999, y1=308.6647, x2=1006.4192, y2=334.2290, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=970.5999, y1=308.6647, x2=1006.4192, y2=334.2290, label=None)], shape=(768, 1360, 3))
48
bbs_aug [BoundingBox(x1=1095.3344, y1=105.0455, x2=1129.6119, y2=142.2751, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1095.3344, y1=105.0455, x2=1129.6119, y2=142.2751, label=None)], shape=(768, 1360, 3))
49
bbs_aug [BoundingBox(x1=591.9945, y1=309.5306, x2=629.6918, y2=336.8986, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=591.9945, y1=309.5306, x2=629.6918, y2=336.8986, label=None)], shape=(768, 1360, 3))
2
/Users/datascience4/Documents/highways-deep-learning/data/A3400_NB1_RAV_R07_180516084515_frame_2447.jpg
0
bbs_aug [BoundingBox(x1=331.1441, y1=45.2581, x2=384.1246, y2=114.5102, label=None)]
type <class 'list'>
Colored image
bbs_aug Bou

bbs_aug [BoundingBox(x1=608.1762, y1=-33.3493, x2=656.5036, y2=33.3311, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=608.1762, y1=-33.3493, x2=656.5036, y2=33.3311, label=None)], shape=(768, 1360, 3))
partly version 2



14
bbs_aug [BoundingBox(x1=324.9452, y1=-133.3631, x2=384.0068, y2=-62.5158, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=324.9452, y1=-133.3631, x2=384.0068, y2=-62.5158, label=None)], shape=(768, 1360, 3))
not in at all



14
bbs_aug [BoundingBox(x1=1.7997, y1=-73.1709, x2=48.6115, y2=-5.0091, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1.7997, y1=-73.1709, x2=48.6115, y2=-5.0091, label=None)], shape=(768, 1360, 3))
not in at all



14
bbs_aug [BoundingBox(x1=230.4489, y1=245.3554, x2=295.4884, y2=318.4203, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=230.4489, y1=245.3554, x2=29

...

type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=976.5159, y1=-36.9971, x2=1049.9347, y2=55.3410, label=None)], shape=(768, 1360, 3))
partly version 2



14
bbs_aug [BoundingBox(x1=1245.2440, y1=328.0374, x2=1324.8685, y2=406.8279, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1245.2440, y1=328.0374, x2=1324.8685, y2=406.8279, label=None)], shape=(768, 1360, 3))
15
bbs_aug [BoundingBox(x1=1286.2773, y1=233.3800, x2=1370.1694, y2=316.0066, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1286.2773, y1=233.3800, x2=1370.1694, y2=316.0066, label=None)], shape=(768, 1360, 3))
partly version 2



15
bbs_aug [BoundingBox(x1=1413.7288, y1=-81.2715, x2=1488.9637, y2=2.6306, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1413.7288, y1=-81.2715, x2=1488.9637, y2=2.6306, label=None)], shape=(768, 1360, 3))
not in at all



15
bbs_aug

bbs_aug [BoundingBox(x1=317.9248, y1=-77.7738, x2=407.0471, y2=8.6470, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=317.9248, y1=-77.7738, x2=407.0471, y2=8.6470, label=None)], shape=(768, 1360, 3))
partly version 2



20
bbs_aug [BoundingBox(x1=1573.4575, y1=191.9954, x2=1657.4235, y2=275.3434, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1573.4575, y1=191.9954, x2=1657.4235, y2=275.3434, label=None)], shape=(768, 1360, 3))
not in at all



20
bbs_aug [BoundingBox(x1=1487.7780, y1=-56.8576, x2=1584.4778, y2=31.9168, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1487.7780, y1=-56.8576, x2=1584.4778, y2=31.9168, label=None)], shape=(768, 1360, 3))
not in at all



20
bbs_aug [BoundingBox(x1=1392.4919, y1=396.5728, x2=1467.1091, y2=491.5599, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1392.4919, y1=

bbs_aug [BoundingBox(x1=1113.3912, y1=228.0499, x2=1182.9974, y2=302.5041, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1113.3912, y1=228.0499, x2=1182.9974, y2=302.5041, label=None)], shape=(768, 1360, 3))
31
bbs_aug [BoundingBox(x1=3.5426, y1=374.8778, x2=67.7526, y2=448.2908, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=3.5426, y1=374.8778, x2=67.7526, y2=448.2908, label=None)], shape=(768, 1360, 3))
32
bbs_aug [BoundingBox(x1=-191.1284, y1=61.6941, x2=-122.6664, y2=127.9532, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=-191.1284, y1=61.6941, x2=-122.6664, y2=127.9532, label=None)], shape=(768, 1360, 3))
not in at all



32
bbs_aug [BoundingBox(x1=-474.8583, y1=-79.8329, x2=-385.9657, y2=7.8185, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=-474.8583, y1=-79.8329, x2=-385.9657, y2=7.8185, label=Non

bbs_aug [BoundingBox(x1=-35.1139, y1=-59.6487, x2=41.4584, y2=16.6847, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=-35.1139, y1=-59.6487, x2=41.4584, y2=16.6847, label=None)], shape=(768, 1360, 3))
partly version 2



41
bbs_aug [BoundingBox(x1=1599.4806, y1=155.6049, x2=1679.0803, y2=245.8858, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1599.4806, y1=155.6049, x2=1679.0803, y2=245.8858, label=None)], shape=(768, 1360, 3))
not in at all



41
bbs_aug [BoundingBox(x1=1193.3021, y1=334.9472, x2=1269.9106, y2=415.1756, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1193.3021, y1=334.9472, x2=1269.9106, y2=415.1756, label=None)], shape=(768, 1360, 3))
42
bbs_aug [BoundingBox(x1=1156.6444, y1=312.8719, x2=1247.5937, y2=398.2008, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1156.6444, y1=312.8719, x2=124

48
bbs_aug [BoundingBox(x1=1651.6181, y1=-69.8165, x2=1725.2331, y2=-1.8496, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1651.6181, y1=-69.8165, x2=1725.2331, y2=-1.8496, label=None)], shape=(768, 1360, 3))
not in at all



48
bbs_aug [BoundingBox(x1=1295.4634, y1=-7.1731, x2=1356.4489, y2=79.7119, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1295.4634, y1=-7.1731, x2=1356.4489, y2=79.7119, label=None)], shape=(768, 1360, 3))
partly version 2



48
bbs_aug [BoundingBox(x1=1111.8872, y1=-244.0073, x2=1190.3608, y2=-133.5995, label=None)]
type <class 'list'>
Colored image
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1111.8872, y1=-244.0073, x2=1190.3608, y2=-133.5995, label=None)], shape=(768, 1360, 3))
not in at all



48
bbs_aug [BoundingBox(x1=1496.2048, y1=158.2120, x2=1578.3609, y2=246.4985, label=None)]
type <class 'list'>
Image is black
bbs_aug BoundingBoxesOnImage([BoundingBox(x1=1496

# The above works but needs improvement
Things now required.
* Bounding boxes are now getting detected inside each image, as well as in the augmented images whether or not they exist in boundaries of that image using the function draw_bbs with padding.
* Even on minor partial images we now need to cut them and redo the augmentations.
* Write a function that only write an xml file and bounding box coordinates if the are not just partially in but fully in.
* If partial or out the augmentation needs to go again until randomly generating one with bounding boxes inside the image. Otherwise it will be hinder the model.
* A function similar to draw_bbs should be able to accomplish this.

* Bounding boxes need to be in the image, if they aren't they should be disgarded and a new image created that until the bounding box sits inside the image and can then be counted.


* Address the category imbalance as well with this work. That if takes the classes, and counts them, that it should then run each for the x amount of classes. related to each type of work.

In [5]:
print(bbs.bounding_boxes)
for bb in bbs.bounding_boxes:
    print(bb)

[BoundingBox(x1=1260.0000, y1=106.0000, x2=1317.0000, y2=172.0000, label=None)]
BoundingBox(x1=1260.0000, y1=106.0000, x2=1317.0000, y2=172.0000, label=None)


In [6]:
image.shape


(768, 1360, 3)